In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('SolarDataProcessed.csv')

In [2]:
'''from sklearn import preprocessing
from sklearn.model_selection import train_test_split
x = df[df.columns[:10]]
y = df['ALLSKY_SFC_SW_DWN']
x_train, x_test, y_train, y_test = train_test_split(x, y , train_size = 0.7, random_state =  90)
#Select numerical columns which needs to be normalized
train_norm = x_train[x_train.columns[0:10]]
test_norm = x_test[x_test.columns[0:10]]
# Normalize Training Data 
std_scale = preprocessing.StandardScaler().fit(train_norm)
x_train_norm = std_scale.transform(train_norm)
#Converting numpy array to dataframe
training_norm_col = pd.DataFrame(x_train_norm, index=train_norm.index, columns=train_norm.columns) 
x_train.update(training_norm_col)
#x_train.head()
# Normalize Testing Data by using mean and SD of training set
x_test_norm = std_scale.transform(test_norm)
testing_norm_col = pd.DataFrame(x_test_norm, index=test_norm.index, columns=test_norm.columns) 
x_test.update(testing_norm_col)'''

"from sklearn import preprocessing\nfrom sklearn.model_selection import train_test_split\nx = df[df.columns[:10]]\ny = df['ALLSKY_SFC_SW_DWN']\nx_train, x_test, y_train, y_test = train_test_split(x, y , train_size = 0.7, random_state =  90)\n#Select numerical columns which needs to be normalized\ntrain_norm = x_train[x_train.columns[0:10]]\ntest_norm = x_test[x_test.columns[0:10]]\n# Normalize Training Data \nstd_scale = preprocessing.StandardScaler().fit(train_norm)\nx_train_norm = std_scale.transform(train_norm)\n#Converting numpy array to dataframe\ntraining_norm_col = pd.DataFrame(x_train_norm, index=train_norm.index, columns=train_norm.columns) \nx_train.update(training_norm_col)\n#x_train.head()\n# Normalize Testing Data by using mean and SD of training set\nx_test_norm = std_scale.transform(test_norm)\ntesting_norm_col = pd.DataFrame(x_test_norm, index=test_norm.index, columns=test_norm.columns) \nx_test.update(testing_norm_col)"

In [3]:
from sklearn.model_selection import train_test_split
x = df[df.columns[:10]]
y = df['ALLSKY_SFC_SW_DWN']
x_train, x_test, y_train, y_test = train_test_split(x, y , train_size = 0.7, random_state =  90)

c:\users\rohan\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
# IMPORT ML CLASSIFIERS
from sklearn.linear_model import LinearRegression # Linear regression
from sklearn.ensemble import RandomForestRegressor # random forest regression
from sklearn.neural_network import MLPRegressor # neural network regression
from sklearn.svm import SVR # support vector regression

from sklearn import preprocessing # ML tools
from sklearn.model_selection import train_test_split # split data


from bokeh.plotting import figure, show, output_notebook

def plot_test(clf,X_test,y_test):
    y_predicted = clf.predict(X_test)

    p = figure(tools='pan,box_zoom,reset',x_range=[0, 100], title='Model validation',y_axis_label='radiation')
    p.grid.minor_grid_line_color = '#eeeeee'

    p.line(range(len(y_test)),y_test,legend='actual',line_color='blue')
    p.line(range(len(y_test)),y_predicted,legend='prediction',line_color='red')
    output_notebook()
    show(p)
    return

def plot_real(clf,x,y_actual,index):
    ''' Plot predictions for actual measurements.
    inputs:
        clf         as classifier   the trained algorithm
        x           as array        timeseries of measurement inputs
        y_actual    as array        corresponding timeseries of actual results
    '''
    y_predicted = clf.predict(x)

    p = figure(toolbar_location='right', title='Predicted vs Actual',y_axis_label='radiation',x_axis_type="datetime")
    p.grid.minor_grid_line_color = '#eeeeee'

    p.line(index,y_actual,legend='actual',line_color='blue')
    p.line(index,y_predicted,legend='prediction',line_color='red')
    output_notebook()
    show(p)
    return

def train_model(x_train, x_test, y_train, y_test,clf,debug=False):
    ''' Train algorithm.
    inputs:
        x       as array        features
        y       as array        label(s)
        clf     as scikit-learn classifier (untrained)
    returns:
        clf     as trained classifier
        accuracy  as float
    '''
    model = clf.fit(x_train,y_train)
    accuracy = clf.score(x_test,y_test)
    return clf, model, accuracy,x_test, y_test

def go(x_train, x_test, y_train, y_test,algorithm,debug=True):
    ''' Easy model train and test. '''
    clf, model, accuracy, x_test, y_test=train_model(x_train, x_test, y_train, y_test,algorithm,debug=True)
    print('Accuracy: %s percent'%str(accuracy*100))

    if debug:
        plot_test(clf,x_test,y_test)
        plot_real(clf,x,y,df.index.values)
    return

def optimize_randomforest(x,y,try_n=10,try_f='auto',try_s=1):
    ''' Find best combo of tunable params for random forest regressor. '''
    best_score = float('-inf') # initialize score
    for n in try_n:
        for f in try_f:
            for s in try_s:
                clf = RandomForestRegressor(oob_score=True,n_estimators=n,max_features=f,min_samples_leaf=s,n_jobs=-1)
                clf.fit(x,y)
                if clf.oob_score_ > best_score:
                    best_score, best_clf, best_n, best_f, best_s = clf.oob_score_, clf, n, f, s
    return clf, best_n, best_f, best_s

n=[100,200,300,500]
f=[2,4,6]
s=[1,2,4,8,16]
clf, n, f, s = optimize_randomforest(x_train,y_train,try_n=n,try_f=f,try_s=s)
print('n_estimators: '+str(n))
print('max_features: '+str(f))
print('min_samples_leaf: '+str(s))
go(x_train, x_test, y_train, y_test,RandomForestRegressor(n_estimators=n,max_features=f,min_samples_leaf=s,n_jobs=-1))
#go(x_train, x_test, y_train, y_test,LinearRegression())

n_estimators: 500
max_features: 6
min_samples_leaf: 1
Accuracy: 99.9995143892188 percent


Loading BokehJS ...

Loading BokehJS ...

In [10]:
import pandas
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
import numpy as np

scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(x)


scores = []
best_svr = SVR(kernel='rbf')
cv = KFold(n_splits=10, random_state=42, shuffle=True)
for train_index, test_index in cv.split(X):
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    best_svr.fit(X_train, y_train)
    scores.append(best_svr.score(X_test, y_test))

Train Index:  [   0    1    2 ... 6605 6606 6607] 

Test Index:  [   8   14   17   23   31   37   50   61   79   80   84   88   91   93
   96  132  156  157  167  179  185  196  198  208  217  228  230  233
  239  247  248  263  265  296  308  318  322  324  332  334  346  351
  373  378  381  393  401  425  432  435  465  468  469  472  476  486
  491  496  503  511  534  535  538  544  549  561  585  604  611  622
  625  676  681  683  696  706  712  736  743  747  752  755  756  763
  764  800  805  811  812  828  842  865  932  960  982  994  996 1002
 1005 1022 1034 1038 1039 1042 1047 1049 1057 1074 1084 1090 1097 1101
 1108 1115 1144 1149 1161 1163 1174 1175 1176 1181 1183 1188 1193 1212
 1223 1242 1253 1263 1297 1321 1339 1345 1352 1354 1357 1362 1374 1385
 1393 1406 1407 1410 1411 1412 1416 1419 1425 1426 1446 1467 1468 1477
 1480 1499 1501 1507 1543 1544 1559 1569 1595 1606 1611 1620 1623 1662
 1670 1671 1695 1703 1706 1726 1737 1765 1768 1786 1789 1805 1807 1820
 1831 1835 1

c:\users\pulkit\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Train Index:  [   0    1    2 ... 6605 6606 6607] 

Test Index:  [  15   19   33   41   44   45   51   63   65   75  101  102  121  122
  124  135  144  149  168  172  177  181  199  216  221  227  231  240
  245  254  257  259  279  290  291  297  303  315  319  321  323  325
  333  347  349  354  371  376  380  408  410  418  420  422  426  437
  439  447  453  462  473  501  506  527  530  533  543  553  555  565
  576  577  586  598  599  620  626  641  652  655  662  672  691  705
  710  730  733  742  746  783  787  794  799  807  808  810  833  834
  864  879  881  893  907  926  957  964  969  977 1010 1018 1027 1032
 1044 1046 1056 1073 1075 1079 1092 1103 1119 1129 1153 1158 1192 1195
 1197 1199 1215 1219 1221 1225 1235 1244 1261 1272 1315 1323 1328 1330
 1335 1340 1360 1370 1371 1375 1397 1400 1405 1414 1420 1421 1433 1452
 1454 1471 1485 1487 1488 1498 1509 1511 1512 1514 1533 1535 1536 1545
 1550 1557 1562 1566 1593 1599 1608 1615 1616 1618 1632 1643 1650 1652
 1656 1657 1

c:\users\pulkit\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Train Index:  [   0    1    2 ... 6605 6606 6607] 

Test Index:  [  12   29   30   47   57   69   71   73   87   90   95  100  106  107
  108  109  110  142  151  169  183  184  188  210  251  272  286  287
  292  293  295  307  336  339  344  367  387  414  415  416  421  429
  438  443  445  452  471  478  485  487  497  505  509  518  548  564
  584  589  605  613  624  627  642  653  654  677  693  701  711  721
  724  734  737  748  751  765  776  782  789  809  829  838  848  856
  861  877  889  896  903  911  915  925  927  931  937  944  947  952
  978 1020 1024 1025 1033 1041 1052 1055 1061 1083 1086 1087 1094 1095
 1109 1112 1117 1121 1128 1142 1168 1170 1172 1173 1186 1194 1203 1204
 1209 1210 1220 1224 1231 1258 1260 1268 1294 1295 1302 1305 1310 1319
 1322 1338 1344 1350 1351 1383 1430 1436 1443 1448 1451 1465 1483 1496
 1504 1513 1520 1539 1553 1554 1558 1561 1564 1575 1578 1580 1586 1600
 1612 1617 1630 1634 1654 1665 1666 1672 1684 1694 1702 1720 1721 1729
 1730 1732 1

...

c:\users\pulkit\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Train Index:  [   0    1    2 ... 6605 6606 6607] 

Test Index:  [   3    5    9   54   55   66   77   94  117  125  126  133  137  164
  201  223  224  232  235  242  249  255  260  280  285  302  320  327
  341  345  357  362  364  375  384  385  388  404  419  455  467  470
  477  499  504  510  523  525  536  539  603  638  645  666  667  671
  673  688  713  716  722  723  732  760  768  770  774  793  797  814
  822  823  827  832  846  872  894  899  901  918  919  922  924  928
  943  946  950  951  953  966  984  989  992 1063 1065 1077 1118 1120
 1141 1145 1155 1156 1160 1169 1191 1202 1205 1214 1217 1248 1250 1252
 1254 1256 1276 1296 1311 1312 1314 1318 1324 1329 1331 1332 1342 1353
 1358 1384 1388 1396 1460 1481 1493 1531 1540 1542 1573 1574 1581 1603
 1604 1605 1619 1633 1639 1645 1646 1668 1673 1680 1719 1724 1753 1759
 1761 1792 1810 1823 1824 1827 1828 1834 1838 1848 1852 1854 1856 1857
 1863 1865 1887 1901 1906 1908 1917 1944 1958 1968 1969 1970 1975 1976
 1984 1990 1

c:\users\pulkit\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Train Index:  [   0    1    2 ... 6605 6606 6607] 

Test Index:  [   4   16   34   64   98  114  116  119  127  130  143  146  154  160
  161  189  190  197  200  202  206  225  241  253  262  301  335  337
  379  391  395  412  417  466  474  488  492  502  512  524  537  559
  562  563  569  574  580  595  600  606  608  616  623  635  646  659
  663  675  698  699  709  717  728  740  753  766  769  775  784  804
  815  825  830  851  853  854  860  863  870  876  878  895  954  956
  959  971  975  980  991  995 1012 1015 1016 1021 1028 1045 1059 1060
 1062 1066 1069 1076 1081 1082 1122 1139 1147 1148 1150 1154 1166 1167
 1184 1218 1243 1267 1275 1282 1291 1300 1306 1348 1363 1365 1369 1409
 1445 1470 1478 1484 1495 1500 1516 1527 1528 1529 1568 1570 1571 1585
 1597 1625 1631 1635 1636 1648 1663 1664 1678 1679 1682 1685 1686 1687
 1693 1696 1707 1715 1722 1733 1734 1757 1794 1802 1806 1816 1819 1841
 1843 1853 1895 1899 1930 1931 1955 1959 1981 1982 1986 1998 2027 2038
 2041 2047 2

c:\users\pulkit\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [11]:
print(np.mean(scores))


0.9311400515228957


In [3]:
df.drop(['MO','DY'], axis = 1, inplace =True)